### Manipulation of PeaskDB de novo-assisted database search results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/102_TROCAS7_Fusion_Apr2021_PEAKS_139/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/102_TROCAS7_Fusion_Apr2021_PEAKS_139


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

 102_TROCAS7_Fusion_Apr2021_PEAKS_139_peptide.csv
 102_TROCAS7_Fusion_Apr2021_PEAKS_139_protein.csv
 102_TROCAS7_Fusion_Apr2021_PEAKS_139_protein.fasta
'DB search psm.csv'
'de novo only peptides.csv'
 peptides.pep.xml
 protein-peptides.csv


In [4]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup102 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/102_TROCAS7_Fusion_Apr2021_PEAKS_139/102_TROCAS7_Fusion_Apr2021_PEAKS_139_peptide.csv")

# remove redundant rows
peaksdb102 = pd.DataFrame.drop_duplicates(peaksdbdup102)

print(peaksdb102.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb102.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb102['#Spec']
del peaksdb102['Accession']
del peaksdb102['PTM']
del peaksdb102['AScore']

mean_length = peaksdb102['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup102))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb102))

#look at the dataframe
peaksdb102.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area Trocas7-102-Bay', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec Trocas7-102-Bay', 'Accession', 'PTM', 'AScore'],
      dtype='object')
11.5
# redundant peaksdb peptides in combined dataframe 60
# nonredundant peaksdb peptides in combined dataframe 60


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,84.01,1881.8350,18,0.4,941.9252,91.55,3960000.0,14,14976,20210407_Trocas7_658_Bay102_DDA_120min_1.raw
1,LGEHNIDVLEGNEQFINAAK,79.91,2210.0967,20,2.5,1106.0583,94.31,21900000.0,14,15485,20210407_Trocas7_658_Bay102_DDA_120min_1.raw
2,IITHPNFNGNTLDNDIM(+15.99)LIK,65.57,2298.1677,20,0.5,767.0636,103.40,779000.0,14,17046,20210407_Trocas7_658_Bay102_DDA_120min_1.raw
3,IITHPNFNGNTLDNDIMLIK,65.25,2282.1729,20,1.2,761.7325,109.09,133000.0,14,17919,20210407_Trocas7_658_Bay102_DDA_120min_1.raw
4,SSGSSYPSLLQC(+57.02)LK,64.82,1525.7446,14,0.0,763.8796,100.09,637000.0,14,16484,20210407_Trocas7_658_Bay102_DDA_120min_1.raw


In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb102['A'] = peaksdb102['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb102['C'] = peaksdb102['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb102['D'] = peaksdb102['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb102['E'] = peaksdb102['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb102['F'] = peaksdb102['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb102['G'] = peaksdb102['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb102['H'] = peaksdb102['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb102 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb102['I'] = peaksdb102['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb102['K'] = peaksdb102['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb102['L'] = peaksdb102['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb102['M'] = peaksdb102['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb102['N'] = peaksdb102['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb102['P'] = peaksdb102['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb102['Q'] = peaksdb102['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb102['R'] = peaksdb102['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb102['S'] = peaksdb102['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb102['T'] = peaksdb102['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb102['V'] = peaksdb102['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb102['W'] = peaksdb102['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb102['Y'] = peaksdb102['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb102['c-carb'] = peaksdb102['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb102['m-oxid'] = peaksdb102['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb102['n-deam'] = peaksdb102['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb102['q-deam'] = peaksdb102['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb102['stripped_peptide'] = peaksdb102['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb102['stripped_length'] = peaksdb102['stripped_peptide'].apply(len)

peaksdb102['NAAF_num.'] = peaksdb102['Area'] / peaksdb102['stripped_length']

# total the number of modifications in sequence
peaksdb102['ptm-total'] = peaksdb102['c-carb'] + peaksdb102['m-oxid'] + peaksdb102['n-deam'] + peaksdb102['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb102['stripped_IL']= peaksdb102['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb102.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/102_BY_T00_GF_PDB/102A_BY_T00_GF_PDB.csv")

# check out the results
peaksdb102.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,84.01,1881.8350,18,0.4,941.9252,91.55,3960000.0,14,14976,...,0,2,0,0,0,SCLISGWGNTK,11,360000.0,2,SCLLSGWGNTK
1,LGEHNIDVLEGNEQFINAAK,79.91,2210.0967,20,2.5,1106.0583,94.31,21900000.0,14,15485,...,0,0,0,0,0,LGEHNIDVLEGNEQFINAAK,20,1095000.0,0,LGEHNLDVLEGNEQFLNAAK
2,IITHPNFNGNTLDNDIM(+15.99)LIK,65.57,2298.1677,20,0.5,767.0636,103.40,779000.0,14,17046,...,0,0,1,0,0,IITHPNFNGNTLDNDIMLIK,20,38950.0,1,LLTHPNFNGNTLDNDLMLLK
3,IITHPNFNGNTLDNDIMLIK,65.25,2282.1729,20,1.2,761.7325,109.09,133000.0,14,17919,...,0,0,0,0,0,IITHPNFNGNTLDNDIMLIK,20,6650.0,0,LLTHPNFNGNTLDNDLMLLK
4,SSGSSYPSLLQC(+57.02)LK,64.82,1525.7446,14,0.0,763.8796,100.09,637000.0,14,16484,...,1,1,0,0,0,SSGSSYPSLLQCLK,14,45500.0,1,SSGSSYPSLLQCLK


In [6]:
# keep only stripped peptide I/L and NAAF
db_102 = peaksdb102[['stripped_IL', 'Area', 'NAAF_num.']]

db_102.set_index('stripped_IL')

# write modified dataframe to new txt file
db_102.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/102A_BY_T00_GF_PDB_ILnaafs.csv")

db_102.head()

,stripped_IL,Area,NAAF_num.
0,SCLLSGWGNTK,3960000.0,360000.0
1,LGEHNLDVLEGNEQFLNAAK,21900000.0,1095000.0
2,LLTHPNFNGNTLDNDLMLLK,779000.0,38950.0
3,LLTHPNFNGNTLDNDLMLLK,133000.0,6650.0
4,SSGSSYPSLLQCLK,637000.0,45500.0


### Exporting txt files of stripped peptides at confidence cutoffs:

In [7]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep102moddup = peaksdb102[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep102dup = peaksdb102[["stripped_peptide"]]

# deduplicate both of these lists
pep102mod = pep102moddup.drop_duplicates()
pep102 = pep102dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 102:", len(pep102moddup))
print("Deduplicated modified PeaksDB peptides in 102:", len(pep102mod))
print("Total stripped PeaksDB peptides in 102:", len(pep102dup))
print("Deduplicated stripped PeaksDB peptides in 102:", len(pep102))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep102.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/102_BY_T00_GF_PDB/102A_BY_T00_GF_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/102_BY_T00_GF_PDB/102A_BY_T00_GF_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/102_BY_T00_GF_PDB/102A_BY_T00_GF_PDB_stripped_peptides.fas

# look at the stripped peptides
pep102.head()

Total modified PeaksDB peptides in 102: 60
Deduplicated modified PeaksDB peptides in 102: 60
Total stripped PeaksDB peptides in 102: 60
Deduplicated stripped PeaksDB peptides in 102: 55


,stripped_peptide
0,SCLISGWGNTK
1,LGEHNIDVLEGNEQFINAAK
2,IITHPNFNGNTLDNDIMLIK
4,SSGSSYPSLLQCLK
8,APVLSDSSCK


## NAAF correction and exporting files with AA and PTM totals:

In [8]:
# made a new dataframe that contains the suMN of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb102['A'].sum(),
        'C': peaksdb102['C'].sum(),
        'D': peaksdb102['D'].sum(),
        'E': peaksdb102['E'].sum(),
        'F': peaksdb102['F'].sum(),
        'G': peaksdb102['G'].sum(),
        'H': peaksdb102['H'].sum(),
        'I': peaksdb102['I'].sum(),
        'K': peaksdb102['K'].sum(),
        'L': peaksdb102['L'].sum(),
        'M': peaksdb102['M'].sum(),
        'N': peaksdb102['N'].sum(),
        'P': peaksdb102['P'].sum(),
        'Q': peaksdb102['Q'].sum(),
        'R': peaksdb102['R'].sum(),
        'S': peaksdb102['S'].sum(),
        'T': peaksdb102['T'].sum(),
        'V': peaksdb102['V'].sum(),
        'W': peaksdb102['W'].sum(),
        'Y': peaksdb102['Y'].sum(),
        'c-carb': peaksdb102['c-carb'].sum(),
        'm-oxid': peaksdb102['m-oxid'].sum(),
        'n-deam': peaksdb102['n-deam'].sum(),
        'q-deam': peaksdb102['q-deam'].sum(),
        'Total area': peaksdb102['Area'].sum(),
        'Total length': peaksdb102['stripped_length'].sum()
       }

totalpeaksdb102 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb102['% C w/ carb'] = totalpeaksdb102['c-carb'] / totalpeaksdb102['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb102['% M w/ oxid'] = totalpeaksdb102['m-oxid'] / totalpeaksdb102['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb102['% N w/ deam'] = totalpeaksdb102['n-deam'] / totalpeaksdb102['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb102['% Q w/ deam'] = totalpeaksdb102['q-deam'] / totalpeaksdb102['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb102['NAAF denom.'] = totalpeaksdb102['Total area'] / totalpeaksdb102['Total length']

# write modified dataframe to new txt file
totalpeaksdb102.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/102_BY_T00_GF_PDB/102A_BY_T00_GF_PDB_totals.csv")

totalpeaksdb102.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,54,10,21,28,14,61,16,48,30,65,...,4,8,4,354270391.0,658,1.0,0.571429,0.153846,0.148148,538404.849544


In [9]:
# use the calculated NAAF factor (in totalpeaksdb102 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 538404.849544

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb102['NAAF factor'] = (peaksdb102['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMN
peaksdb102_NAAF = peaksdb102[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb102_NAAF['A-NAAF20'] = peaksdb102_NAAF['A'] * peaksdb102['NAAF factor']
peaksdb102_NAAF['C-NAAF20'] = peaksdb102_NAAF['C'] * peaksdb102['NAAF factor']
peaksdb102_NAAF['D-NAAF20'] = peaksdb102_NAAF['D'] * peaksdb102['NAAF factor']
peaksdb102_NAAF['E-NAAF20'] = peaksdb102_NAAF['E'] * peaksdb102['NAAF factor']
peaksdb102_NAAF['F-NAAF20'] = peaksdb102_NAAF['F'] * peaksdb102['NAAF factor']
peaksdb102_NAAF['G-NAAF20'] = peaksdb102_NAAF['G'] * peaksdb102['NAAF factor']
peaksdb102_NAAF['H-NAAF20'] = peaksdb102_NAAF['H'] * peaksdb102['NAAF factor']
peaksdb102_NAAF['I-NAAF20'] = peaksdb102_NAAF['I'] * peaksdb102['NAAF factor']
peaksdb102_NAAF['K-NAAF20'] = peaksdb102_NAAF['K'] * peaksdb102['NAAF factor']
peaksdb102_NAAF['L-NAAF20'] = peaksdb102_NAAF['L'] * peaksdb102['NAAF factor']
peaksdb102_NAAF['M-NAAF20'] = peaksdb102_NAAF['M'] * peaksdb102['NAAF factor']
peaksdb102_NAAF['N-NAAF20'] = peaksdb102_NAAF['N'] * peaksdb102['NAAF factor']
peaksdb102_NAAF['P-NAAF20'] = peaksdb102_NAAF['P'] * peaksdb102['NAAF factor']
peaksdb102_NAAF['Q-NAAF20'] = peaksdb102_NAAF['Q'] * peaksdb102['NAAF factor']
peaksdb102_NAAF['R-NAAF20'] = peaksdb102_NAAF['R'] * peaksdb102['NAAF factor']
peaksdb102_NAAF['S-NAAF20'] = peaksdb102_NAAF['S'] * peaksdb102['NAAF factor']
peaksdb102_NAAF['T-NAAF20'] = peaksdb102_NAAF['T'] * peaksdb102['NAAF factor']
peaksdb102_NAAF['V-NAAF20'] = peaksdb102_NAAF['V'] * peaksdb102['NAAF factor']
peaksdb102_NAAF['W-NAAF20'] = peaksdb102_NAAF['W'] * peaksdb102['NAAF factor']
peaksdb102_NAAF['Y-NAAF20'] = peaksdb102_NAAF['Y'] * peaksdb102['NAAF factor']

# multiply the NAAF20 factor by the PTMN normalize its abundance by peak area and peptide length

peaksdb102_NAAF['ccarb-NAAF20'] = peaksdb102_NAAF['c-carb'] * peaksdb102_NAAF['NAAF factor']
peaksdb102_NAAF['moxid-NAAF20'] = peaksdb102_NAAF['m-oxid'] * peaksdb102_NAAF['NAAF factor']
peaksdb102_NAAF['ndeam-NAAF20'] = peaksdb102_NAAF['n-deam'] * peaksdb102_NAAF['NAAF factor']
peaksdb102_NAAF['qdeam-NAAF20'] = peaksdb102_NAAF['q-deam'] * peaksdb102_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb102_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/102_BY_T00_GF_PDB/102A_BY_T00_GF_PDB_naaf.csv")

peaksdb102_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,SCLISGWGNTK,0.668642,3,2,0,1,0,3,0,1,...,0.0,1.337284,1.337284,0.000000,0.668642,0.000000,1.337284,0.000000,0.0,0.0
1,LGEHNIDVLEGNEQFINAAK,2.033786,2,0,1,3,1,2,1,2,...,0.0,0.000000,0.000000,2.033786,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,IITHPNFNGNTLDNDIMLIK,0.072343,0,0,2,0,1,1,1,4,...,0.0,0.000000,0.144687,0.000000,0.000000,0.000000,0.000000,0.072343,0.0,0.0
3,IITHPNFNGNTLDNDIMLIK,0.012351,0,0,2,0,1,1,1,4,...,0.0,0.000000,0.024703,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,SSGSSYPSLLQCLK,0.084509,0,1,0,0,0,1,0,0,...,0.0,0.422544,0.000000,0.000000,0.000000,0.084509,0.084509,0.000000,0.0,0.0


In [10]:
# made a new dataframe that contains the suMN of NAAF normalized AAs for peaksdb102 peaksdb results
# also contains the suMN of the NAAF-corrected PTMN occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb102_NAAF['NAAF factor'].sum(),
        'A': peaksdb102_NAAF['A-NAAF20'].sum(),
        'C': peaksdb102_NAAF['C-NAAF20'].sum(),
        'D': peaksdb102_NAAF['D-NAAF20'].sum(),
        'E': peaksdb102_NAAF['E-NAAF20'].sum(),
        'F': peaksdb102_NAAF['F-NAAF20'].sum(),
        'G': peaksdb102_NAAF['G-NAAF20'].sum(),
        'H': peaksdb102_NAAF['H-NAAF20'].sum(),
        'I': peaksdb102_NAAF['I-NAAF20'].sum(),
        'K': peaksdb102_NAAF['K-NAAF20'].sum(),
        'L': peaksdb102_NAAF['L-NAAF20'].sum(),
        'M': peaksdb102_NAAF['M-NAAF20'].sum(),
        'N': peaksdb102_NAAF['N-NAAF20'].sum(),
        'P': peaksdb102_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb102_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb102_NAAF['R-NAAF20'].sum(),
        'S': peaksdb102_NAAF['S-NAAF20'].sum(),
        'T': peaksdb102_NAAF['T-NAAF20'].sum(),
        'V': peaksdb102_NAAF['V-NAAF20'].sum(),
        'W': peaksdb102_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb102_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb102_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb102_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb102_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb102_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb102_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb102_NAAF['% C w/ carb.'] = totalpeaksdb102_NAAF['c-carb'] / totalpeaksdb102_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb102_NAAF['% M w/ oxid'] = totalpeaksdb102_NAAF['m-oxid'] / totalpeaksdb102_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb102_NAAF['% N w/ deam'] = totalpeaksdb102_NAAF['n-deam'] / totalpeaksdb102_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb102_NAAF['% Q w/ deam'] = totalpeaksdb102_NAAF['q-deam'] / totalpeaksdb102_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb102_NAAF['NAAF check'] = totalpeaksdb102_NAAF['NAAF'] / 538404.849544

# write modified dataframe to new txt file, same name + totals
totalpeaksdb102_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/102_BY_T00_GF_PDB/102A_BY_T00_GF_PDB_naaf_totals.csv")

totalpeaksdb102_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,73.953522,53.339131,2.705925,2.939905,9.790768,2.800651,10.904842,2.864775,8.337957,3.428026,...,2.811194,2.705925,0.143539,0.927382,0.162716,1.0,0.827682,0.041475,0.037221,0.000137
